## Create US Core Patient Resource Using the Python FHIRClient Module

### Prerequisites:

- Python 3.6 or greater
    for instruction how to install Python and modules and optionally setting up atom as a python IDE and creating virtual environments, If you are new to Python or need help getting starged I recommend these YouYube videos by Corey Schaefer:
    
[![image.jpg](http://img.youtube.com/vi/YYXdXT2l-Gg/sddefault.jpg)](https://www.youtube.com/watch?v=YYXdXT2l-Gg)

[![image.jpg](http://img.youtube.com/vi/CqvZ3vGoGs0/sddefault.jpg)](https://www.youtube.com/watch?v=CqvZ3vGoGs0)

[![image.jpg](http://img.youtube.com/vi/DjEuROpsvp4/sddefault.jpg)](https://www.youtube.com/watch?v=DjEuROpsvp4)

[![image.jpg](http://img.youtube.com/vi/zDYL22QNiWk/sddefault.jpg)](https://www.youtube.com/watch?v=zDYL22QNiWk)


- installed modules:
   - [fhirclient](https://github.com/smart-on-fhir/client-py)  ( FHIR version STU3 )
   - [requests](docs.python-requests.org/en/master/)
   - [csv — CSV File Reading and Writing](https://docs.python.org/3/library/csv.html)
   
### Introduction:
 
IN the code below, I walk through the steps to create US Core Patient resource, validate it, Post it to a FHIR server and then fetch it in xml format.  The data is based off of a simple excel file which serves as a data entry form.  To simplify things the worksheet is saved as a CSV file which is used directly in the code below.  

See the FHIRClient documentation for a quick overview how to use the fhirclient models.  The models are instantiated with a Python dictionary structure or primitives types ( str, int, float, or bool ).   Repeating elements are represented at lists.  Also the FHIRClient models perform some data validation as well and provide for a more `pythonic` way to represent FHIR resources.
 

#### First import all the necessary modules

We will be using all these FHIRClient models ( classes ) for this examples and they include the patient resource models and all the datatype classes used by the resource.  The `IPython` and `pandas` modules are used to make this Jupyter Notebook display pretty.

In [4]:
# ******************** modules **************************

import fhirclient.models.patient as P
import fhirclient.models.identifier as I
import fhirclient.models.coding as C
import fhirclient.models.codeableconcept as CC
import fhirclient.models.fhirdate as D
import fhirclient.models.extension as X
import fhirclient.models.humanname as H
import fhirclient.models.meta as M
import fhirclient.models.valueset as VS
from json import dumps
from requests import get, post, put
import os
from csv import DictReader
from IPython.display import display, Markdown
from pandas import read_csv

####  Assign Global Variables

Here is where we assign all the global varaibles for this example such as the local paths for file input and output which need to
be updated for your local environment.

In [5]:
# ******************** globals **************************
in_path =r'C:\Users\Eric\Documents\Jan_2019_FHIR_Experience\US_CorePerson_data.csv'
# out_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
out_path = r'C:\Users\Eric\Documents\Jan_2019_FHIR_Experience'
fhir_term_server = 'http://test.fhir.org/r3'
#fhir_test_server = 'http://test.fhir.org/r3'
fhir_test_server = 'http://wildfhir.aegis.net/fhir3-0-1'
headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}
profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient



####  Defined functions

For this example I tried to make a simple script but there is one function defined below which demonstrates how to call a FHIR terminology server to fetch a code display and code system  given a code and a Value Set.  It is used for the Race codes as seen below.  This uses the Requests module to perform the `$expand` operation on a ValueSet.

In [6]:
# *********************  code lookup function **********************

def code_lookup(code, vsid):  # look up code display and system based on code value using term server and $expansion operation
    #GET [base]/CodeSystem/$lookup?system=http://loinc.org&code=1963-8
    params = {
    'filter': code
    }
    #   r = requests.get('https://httpbin.org/post', params = {'key':'value'}, header = {'key':'value'})
    r = get(f'{fhir_term_server}/ValueSet/{vsid}/$expand', params = params, headers = headers)
    # print(r.status_code)
    if r.status_code < 300:
        vs = VS.ValueSet(r.json())
        return {'display': vs.expansion.contains[0].display, 'system': vs.expansion.contains[0].system
            }
    else:
        print (dumps(r.json(),indent=3))



####  Instanatiating All the Classes

Before we start assigning patient data to the Patient Class, The classes are instantatiated.  See [this video](https://www.youtube.com/watch?v=ZDa-Z5JzLYM) for more
information about Python classes and instances.  For the FHIRClient Python models, the classes are represented as Python dictionary structures using the method `as_json()`

In [7]:
#********************* instanatiate classes **********************

patient = P.Patient()
patientcommunication = P.PatientCommunication()
meta = M.Meta()
identifier = I.Identifier()
name = H.HumanName()
birthdate = D.FHIRDate()
language = CC.CodeableConcept()
coding = C.Coding()
race = C.Coding()
ethn = C.Coding()
b_sex_extension = X.Extension()
race_extension = X.Extension()
race_ombcat_extension = X.Extension()
race_ombdet_extension = X.Extension()
race_text_extension = X.Extension()
ethn_extension = X.Extension()
ethn_ombcat_extension = X.Extension()
ethn_ombdet_extension = X.Extension()
ethn_text_extension = X.Extension()


display(Markdown(f'**Instantiated Python FHIR Patient Class as a Python dictionary object:** `{patient.as_json()}`'))


**Instantiated Python FHIR Patient Class as a Python dictionary object:** `{'resourceType': 'Patient'}`

#### Get patient data

To populate the patient resource, the data is entered from some source.  The datasource may be a database, spreadsheet, manual entry
(e.g. from a form).  In this example, data is entered in a spreadsheet form and saved as a CSV file to make things easier when
it comes to data type conversion, since with a csv -everything is a string.

Notice that there are several fields for race and ethnicity.  In this example only a single race and ethnicity category are used.

In [8]:
#********************* get patient data **********************
# get data from form ( csv file )
df1 = read_csv(in_path)

display(Markdown('**Get data from csv file**'))

df1


**Get data from csv file**

,row,mrn,f_name,l_name,mi,b_sex,gender,b_date,lang,race_cat_1,race_cat_2,race_cat_3,race_det_1,race_det_2,ethn_cat,ethn_det_1,ethn_det_2
0,1000,1032702,Amy,Shaw,"V,",F,female,2007-02-20,en|English,2106-3|White,1002-5|American Indian or Alaska Native,2028-9|Asian,1586-7|Shoshone,2036-2|Filipino,Hispanic or Latino,2184-0|Dominican,2148-5|Mexican
1,1001,1798520,Harold,Thalburg,G,M,male,1964-06-19,en|English,2106-3|White,NaN,NaN,2116-2|Scottish,NaN,Non Hispanic or Latino,NaN,NaN
2,1002,1827981,Josef,Banks,E,M,male,1954-01-03,de-AT|German (Austria),ASKU|Asked but no answer,NaN,NaN,NaN,NaN,Non Hispanic or Latino,NaN,NaN
3,1003,1234568,Marie,Espinoza,NaN,F,female,1980-02-02,es|Spanish,2106-3|White,NaN,NaN,NaN,NaN,Hispanic or Latino,2180-8|Puerto Rican,NaN
4,1004,7855521,Sofia,Juarez,NaN,F,female,1986-08-10,es|Spanish,2106-3|White,NaN,NaN,NaN,NaN,Hispanic or Latino,2148-5|Mexican,NaN
5,1005,1258622,An,Nguyen,P,M,male,1990-10-15,en|English,2028-9|Asian,NaN,NaN,2047-9|Vietnamese,NaN,Non Hispanic or Latino,NaN,NaN


In [9]:
with open(in_path, mode='r',encoding='utf-8-sig') as csv_file:
    csv_reader = DictReader(csv_file)
    for data in csv_reader:

        display(Markdown (f'**CSV file data for row = {data["row"]}**'))
 
        for k,v in data.items():
            print(f'{k} = {v}')

**CSV file data for row = 1000**

row = 1000
mrn = 1032702
f_name = Amy
l_name = Shaw
mi = V,
b_sex = F
gender = female
b_date = 2007-02-20
lang = en|English
race_cat_1 = 2106-3|White
race_cat_2 = 1002-5|American Indian or Alaska Native
race_cat_3 = 2028-9|Asian
race_det_1 = 1586-7|Shoshone
race_det_2 = 2036-2|Filipino
ethn_cat = Hispanic or Latino
ethn_det_1 = 2184-0|Dominican
ethn_det_2 = 2148-5|Mexican


**CSV file data for row = 1001**

row = 1001
mrn = 1798520
f_name = Harold
l_name = Thalburg
mi = G
b_sex = M
gender = male
b_date = 1964-06-19
lang = en|English
race_cat_1 = 2106-3|White
race_cat_2 = 
race_cat_3 = 
race_det_1 = 2116-2|Scottish
race_det_2 = 
ethn_cat = Non Hispanic or Latino
ethn_det_1 = 
ethn_det_2 = 


**CSV file data for row = 1002**

...

row = 1003
mrn = 1234568
f_name = Marie
l_name = Espinoza
mi = 
b_sex = F
gender = female
b_date = 1980-02-02
lang = es|Spanish
race_cat_1 = 2106-3|White
race_cat_2 = 
race_cat_3 = 
race_det_1 = 
race_det_2 = 
ethn_cat = Hispanic or Latino
ethn_det_1 = 2180-8|Puerto Rican
ethn_det_2 = 


**CSV file data for row = 1004**

row = 1004
mrn = 7855521
f_name = Sofia
l_name = Juarez
mi = 
b_sex = F
gender = female
b_date = 1986-08-10
lang = es|Spanish
race_cat_1 = 2106-3|White
race_cat_2 = 
race_cat_3 = 
race_det_1 = 
race_det_2 = 
ethn_cat = Hispanic or Latino
ethn_det_1 = 2148-5|Mexican
ethn_det_2 = 


**CSV file data for row = 1005**

row = 1005
mrn = 1258622
f_name = An
l_name = Nguyen
mi = P
b_sex = M
gender = male
b_date = 1990-10-15
lang = en|English
race_cat_1 = 2028-9|Asian
race_cat_2 = 
race_cat_3 = 
race_det_1 = 2047-9|Vietnamese
race_det_2 = 
ethn_cat = Non Hispanic or Latino
ethn_det_1 = 
ethn_det_2 = 


#### Create Patient resource

After getting the source data, the patient resource can be populated.   The following code cells shows step by step how the different data types are assigned using the FHIRClient model.

#####  Assign values to the simple id and active elements

In [10]:
                #********************* main create patient **********************

                # create simple id and active elements
                patient.id = f'jan-2019-fhir-experience-example-{data["row"]}'
                patient.active = True
                
                display(Markdown('**Python FHIR Patient Class is growing as the csv data are assigned to its attributes ...**'))
                display(Markdown(f'`{patient.as_json()}`'))


**Python FHIR Patient Class is growing as the csv data are assigned to its attributes ...**

`{'id': 'jan-2019-fhir-experience-example-1005', 'active': True, 'resourceType': 'Patient'}`

#####  Assign values to complex `meta` element 
 
 The meta class contains a repeating `profile` elment

In [11]:
                # create complex meta element for  (repeating) profile element
                meta.profile = []  # initialize repeating elements as empty list
                meta.profile.append(profile) # append object to repeating elements
                patient.meta = meta
                
                display(Markdown('**add meta element ...**'))
                display(Markdown(f'`{patient.as_json()}`'))


**add meta element ...**

`{'id': 'jan-2019-fhir-experience-example-1005', 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']}, 'active': True, 'resourceType': 'Patient'}`

#####  Assign values to the complex identifier element

Note in this example only a identifier is provided.

In [12]:
                # create complex identifier element
                identifier.system = 'http://example.org/pids'
                identifier.value = data['mrn']
                patient.identifier=[]   # initialize repeating elements as empty list
                patient.identifier.append(identifier) # append object to repeating elements
                
                display(Markdown('**add identifier ...**'))
                display(Markdown(f'`{patient.as_json()}`'))


**add identifier ...**

`{'id': 'jan-2019-fhir-experience-example-1005', 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']}, 'active': True, 'identifier': [{'system': 'http://example.org/pids', 'value': '1258622'}], 'resourceType': 'Patient'}`

#####  Assign values to the complex repeating name element

Note in this example only a single name is provided.

In [13]:
                # create complex repeating name element
                name.family = data['l_name']
                name.given = []
                name.given.append(data['f_name'])
                if data['mi']: name.given.append(data['mi'])
                patient.name=[]
                patient.name.append(name)
                
                display(Markdown('**add name ...**'))
                display(Markdown(f'`{patient.as_json()}`'))


**add name ...**

`{'id': 'jan-2019-fhir-experience-example-1005', 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']}, 'active': True, 'identifier': [{'system': 'http://example.org/pids', 'value': '1258622'}], 'name': [{'family': 'Nguyen', 'given': ['An', 'P']}], 'resourceType': 'Patient'}`

#####   Assign simple `gender` and `birthdate` elements

Birthdate is treated as an iso string in Python.

In [14]:
                #create simple gender and birthdate element ( birthdate is treated as iso string in Python)
                patient.gender = data['gender']
                b_date = D.FHIRDate(data['b_date'])
                patient.birthDate = b_date
                
                display(Markdown('**add gender and birthdate ...**'))
                display(Markdown(f'`{patient.as_json()}`'))


**add gender and birthdate ...**

`{'id': 'jan-2019-fhir-experience-example-1005', 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']}, 'active': True, 'birthDate': '1990-10-15', 'gender': 'male', 'identifier': [{'system': 'http://example.org/pids', 'value': '1258622'}], 'name': [{'family': 'Nguyen', 'given': ['An', 'P']}], 'resourceType': 'Patient'}`

#####  Assign backbone `communication` element 

`CodeableConcept` and `Coding` datatypes are used in this element.  This pattern is repeated for other coded elements below.

In [15]:
                # create backbone communication element with codeableconcept
                patient.communication = []
                lang_code = data['lang'].split('|')[0]  # split data into code and display
                lang_dis = data['lang'].split('|')[1]
                language.coding = []
                coding.code = lang_code
                coding.system = 'urn:ietf:bcp:47'  
                coding.display = lang_dis
                language.coding.append(coding)
                language.text = lang_dis
                patientcommunication.language = language
                patient.communication.append(patientcommunication)
                
                display(Markdown('**add communication element ...**'))
                display(Markdown(f'`{patient.as_json()}`'))


**add communication element ...**

`{'id': 'jan-2019-fhir-experience-example-1005', 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']}, 'active': True, 'birthDate': '1990-10-15', 'communication': [{'language': {'coding': [{'code': 'en', 'display': 'English', 'system': 'urn:ietf:bcp:47'}], 'text': 'English'}}], 'gender': 'male', 'identifier': [{'system': 'http://example.org/pids', 'value': '1258622'}], 'name': [{'family': 'Nguyen', 'given': ['An', 'P']}], 'resourceType': 'Patient'}`

#####  Assign values to the extension

There are three extensions defined for the US-Core Patient.  The "Birth Sex" extension is a simple extension and the "Race" and "Ethnicity" extensions are complex extensions.

######  Assign values to the simple "Birth Sex" extension

For each simple extension, a url and a value ( in this case a code ) are assigned.

In [16]:
                # create simple birth sex (repeating) extension
                patient.extension = []
                b_sex_extension.url = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex'  # canonical url for birthsex is 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex'
                b_sex_extension.valueCode = data['b_sex']
                patient.extension.append(b_sex_extension)
                
                display(Markdown('**add US Core Extensions starting with birth_sex ...**'))
                display(Markdown(f'`{patient.as_json()}`'))


**add US Core Extensions starting with birth_sex ...**

`{'id': 'jan-2019-fhir-experience-example-1005', 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']}, 'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex', 'valueCode': 'M'}], 'active': True, 'birthDate': '1990-10-15', 'communication': [{'language': {'coding': [{'code': 'en', 'display': 'English', 'system': 'urn:ietf:bcp:47'}], 'text': 'English'}}], 'gender': 'male', 'identifier': [{'system': 'http://example.org/pids', 'value': '1258622'}], 'name': [{'family': 'Nguyen', 'given': ['An', 'P']}], 'resourceType': 'Patient'}`

###### Assign values to the simple "Birth Sex" extension

For each complex extension, a url and nested extensions and a values ( in this case a CodeableConcepts and strings ) are assigned.

For this example, a FHIR terminology server to fetch a code display and code system for the race code and a Value Set.  This uses the Requests module to perform the `$expand` operation on a ValueSet.

In [17]:
                
                # create complex race  (repeating) extension using codeableconcept
        
                race_cat_code = data['race_cat_1'].split('|')[0]
                code_meta = code_lookup(race_cat_code, 'omb-race-category')  # look up using terminology server
                sys = code_meta['system']
                dis = code_meta['display']
                
                
                display(Markdown('**Get code system and code display for race from terminology server:**'))
                print(f'sys = {sys}')
                print(f'dis = {dis}')
            

**Get code system and code display for race from terminology server:**

sys = urn:oid:2.16.840.1.113883.6.238
dis = Asian


After getting the code system and display for the code, the extension is populated.

In [18]:
                race_extension.url = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'  # canonical url for race
                race_extension.extension = []
                race_text_extension.url = 'text'  # url for "subextension"
                race_text_extension.valueString = dis  # use display
                race_extension.extension.append(race_text_extension)

                race_ombcat_extension.url = 'ombCategory'  # url for "subextension"
                race.system = sys  # look up using terminology server
                race.code = race_cat_code
                race.display = dis  # look up using terminology server
                race_ombcat_extension.valueCoding = race
                race_extension.extension.append(race_ombcat_extension)

                patient.extension.append(race_extension)
                
                display(Markdown('**add US Core Race Extension ...**'))
                display(Markdown(f'`{patient.as_json()}`'))



**add US Core Race Extension ...**

`{'id': 'jan-2019-fhir-experience-example-1005', 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']}, 'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex', 'valueCode': 'M'}, {'extension': [{'url': 'text', 'valueString': 'Asian'}, {'url': 'ombCategory', 'valueCoding': {'code': '2028-9', 'display': 'Asian', 'system': 'urn:oid:2.16.840.1.113883.6.238'}}], 'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'}], 'active': True, 'birthDate': '1990-10-15', 'communication': [{'language': {'coding': [{'code': 'en', 'display': 'English', 'system': 'urn:ietf:bcp:47'}], 'text': 'English'}}], 'gender': 'male', 'identifier': [{'system': 'http://example.org/pids', 'value': '1258622'}], 'name': [{'family': 'Nguyen', 'given': ['An', 'P']}], 'resourceType': 'Patient'}`

Finally to complete this example , the ehtnicity extension is populated with a similar pattern to the race extension above

In [19]:
                # create complex ethnicity (repeating) extension using codeableconcept

                ethn_extension.url = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'  # canonical url for ethn
                ethn_extension.extension = []

                ethn_text_extension.url = 'text'  # url for "subextension"
                ethn_text_extension.valueString = dis  # use display
                ethn_extension.extension.append(ethn_text_extension)

                ethn_ombcat_extension.url = 'ombCategory'  # url for "subextension"
                ethn.system = 'urn:oid:2.16.840.1.113883.6.238'
                ethn.code = '2135-2' if data['ethn_cat'] == 'Hispanic or Latino' else '2186-5'
                ethn.display = data['ethn_cat']
                ethn_ombcat_extension.valueCoding = ethn
                ethn_extension.extension.append(ethn_ombcat_extension)

                patient.extension.append(ethn_extension)
                
                display(Markdown('**and finally add US Core Ehtnicity Extension ...**'))
                display(Markdown(f'`{patient.as_json()}`'))



**and finally add US Core Ehtnicity Extension ...**

`{'id': 'jan-2019-fhir-experience-example-1005', 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']}, 'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex', 'valueCode': 'M'}, {'extension': [{'url': 'text', 'valueString': 'Asian'}, {'url': 'ombCategory', 'valueCoding': {'code': '2028-9', 'display': 'Asian', 'system': 'urn:oid:2.16.840.1.113883.6.238'}}], 'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race'}, {'extension': [{'url': 'text', 'valueString': 'Asian'}, {'url': 'ombCategory', 'valueCoding': {'code': '2186-5', 'display': 'Non Hispanic or Latino', 'system': 'urn:oid:2.16.840.1.113883.6.238'}}], 'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity'}], 'active': True, 'birthDate': '1990-10-15', 'communication': [{'language': {'coding': [{'code': 'en', 'display': 'English', 'system': 'urn:ietf:bcp:47'}], 'text': 'English'}}], 'gender': 'male', 'identifier': [{'system': 'http://example.org/pids', 'value': '1258622'}], 'name': [{'family': 'Nguyen', 'given': ['An', 'P']}], 'resourceType': 'Patient'}`

#### Pretty Print the resource as a json string

In Python the resource is represented as a 'dict' type.  to share this data it needs to be converted to JSON format using the Python standard library methods from the "json" module

In [20]:
                # pretty print the resource as a json string  ( in Python is a 'dict' type)


                display(Markdown('**Pretty print the resource as a json string ...**'))
                display(Markdown(f'`{dumps(patient.as_json(),indent =3)}`'))


**Pretty print the resource as a json string ...**

`{
   "id": "jan-2019-fhir-experience-example-1005",
   "meta": {
      "profile": [
         "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"
      ]
   },
   "extension": [
      {
         "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex",
         "valueCode": "M"
      },
      {
         "extension": [
            {
               "url": "text",
               "valueString": "Asian"
            },
            {
               "url": "ombCategory",
               "valueCoding": {
                  "code": "2028-9",
                  "display": "Asian",
                  "system": "urn:oid:2.16.840.1.113883.6.238"
               }
            }
         ],
         "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-race"
      },
      {
         "extension": [
            {
               "url": "text",
               "valueString": "Asian"
            },
            {
               "url": "ombCategory",
               "valueCoding": {
                  "code": "2186-5",
                  "display": "Non Hispanic or Latino",
                  "system": "urn:oid:2.16.840.1.113883.6.238"
               }
            }
         ],
         "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity"
      }
   ],
   "active": true,
   "birthDate": "1990-10-15",
   "communication": [
      {
         "language": {
            "coding": [
               {
                  "code": "en",
                  "display": "English",
                  "system": "urn:ietf:bcp:47"
               }
            ],
            "text": "English"
         }
      }
   ],
   "gender": "male",
   "identifier": [
      {
         "system": "http://example.org/pids",
         "value": "1258622"
      }
   ],
   "name": [
      {
         "family": "Nguyen",
         "given": [
            "An",
            "P"
         ]
      }
   ],
   "resourceType": "Patient"
}`

####  Validate the Resource

Using the [$validate](http://build.fhir.org/resource-operation-validate.html) operation, the example is validated 
by a FHIR Reference Server.  The results are displayed below in the human readable text as xhtml.

In [30]:
                # *********************** validate Resource ********************************

                params = {
                'profile': profile
                }
                #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
                r = post(f'{fhir_test_server}/Patient/$validate', params = params, headers = headers, data = dumps(patient.as_json()))
                # print(r.status_code)
                # view  output
                display(Markdown(f'<h1>Validation output</h1>{r.json()["text"]["div"]}'))


<h1>Validation output</h1><div xmlns="http://www.w3.org/1999/xhtml"><table class="grid"><tr><td><b>Severity</b></td><td><b>Location</b></td><td><b>Code</b></td><td><b>Details</b></td><td><b>Diagnostics</b></td></tr><tr><td>INFORMATION</td><td/><td>Informational Note</td><td>Validation successful, no issues found</td><td>Validation successful, no issues found</td></tr></table></div>

##### PUT Resource on FHIR Server

Using the FHIR RESTful `PUT` transaction, the resource is loaded onto the FHIR test server. The results are displayed below in the human readable text as xhtml.

In [27]:
                # *********************** PUT Resource on server********************************

                #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
                r = put(f'{fhir_test_server}/Patient/{patient.id}', headers = headers, data = dumps(patient.as_json()))

                display(Markdown(f'<h1>Put Response</h1><p><strong>Transaction Status</strong> = {r.status_code}</p>{r.json()["text"]["div"]}<p>raw output :</p><code>{dumps(r.json(),indent=3)}</code>'))


<h1>Put Response</h1><p><strong>Transaction Status</strong> = 200</p><div xmlns="http://www.w3.org/1999/xhtml"><p><b>Generated Narrative with Details</b></p><p><b>identifier</b>: 1258622      
      </p><p><b>active</b>: true      
      </p><p><b>name</b>: An P Nguyen       
      </p><p><b>gender</b>: MALE      
      </p><p><b>birthDate</b>: Oct 15, 1990      
      </p><h3>Communications</h3><table class="grid"><tr><td>-</td><td><b>Language</b></td></tr><tr><td>*</td><td>English             
            <span style="background: LightGoldenRodYellow">(Details : {urn:ietf:bcp:47 code 'en' = 'en', given as 'English'})</span></td></tr></table></div><p>raw output :</p><code>{
   "resourceType": "Patient",
   "id": "jan-2019-fhir-experience-example-1005",
   "meta": {
      "versionId": "10",
      "lastUpdated": "2019-01-07T19:54:48.029-05:00",
      "profile": [
         "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"
      ]
   },
   "text": {
      "status": "generated",
      "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Generated Narrative with Details</b></p><p><b>identifier</b>: 1258622      \n      </p><p><b>active</b>: true      \n      </p><p><b>name</b>: An P Nguyen       \n      </p><p><b>gender</b>: MALE      \n      </p><p><b>birthDate</b>: Oct 15, 1990      \n      </p><h3>Communications</h3><table class=\"grid\"><tr><td>-</td><td><b>Language</b></td></tr><tr><td>*</td><td>English             \n            <span style=\"background: LightGoldenRodYellow\">(Details : {urn:ietf:bcp:47 code 'en' = 'en', given as 'English'})</span></td></tr></table></div>"
   },
   "extension": [
      {
         "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex",
         "valueCode": "M"
      },
      {
         "extension": [
            {
               "url": "text",
               "valueString": "Asian"
            },
            {
               "url": "ombCategory",
               "valueCoding": {
                  "system": "urn:oid:2.16.840.1.113883.6.238",
                  "code": "2028-9",
                  "display": "Asian"
               }
            }
         ],
         "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-race"
      },
      {
         "extension": [
            {
               "url": "text",
               "valueString": "Asian"
            },
            {
               "url": "ombCategory",
               "valueCoding": {
                  "system": "urn:oid:2.16.840.1.113883.6.238",
                  "code": "2186-5",
                  "display": "Non Hispanic or Latino"
               }
            }
         ],
         "url": "http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity"
      }
   ],
   "identifier": [
      {
         "system": "http://example.org/pids",
         "value": "1258622"
      }
   ],
   "active": true,
   "name": [
      {
         "family": "Nguyen",
         "given": [
            "An",
            "P"
         ]
      }
   ],
   "gender": "male",
   "birthDate": "1990-10-15",
   "communication": [
      {
         "language": {
            "coding": [
               {
                  "system": "urn:ietf:bcp:47",
                  "code": "en",
                  "display": "English"
               }
            ],
            "text": "English"
         }
      }
   ]
}</code>

##### GET Resource from FHIR Server in XML format

Using the FHIR RESTful `GET` transaction, fetch the resource from the FHIR test server. The results are displayed below  human  xml.

In [36]:
                # *********************** GET Resource as xml from server********************************
                headers_xml = {
                'Accept':'application/fhir+xml',
                'Content-Type':'application/fhir+xml'
                }

                #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
                r = get(f'{fhir_test_server}/Patient/{patient.id}', headers = headers_xml)

                display(Markdown(f'<h1>Get Resource as XML</h1><p><strong>Transaction Status</strong> = {r.status_code}</p><textarea rows="50" cols="79">{r.text}</textarea>'))


<h1>Get Resource as XML</h1><p><strong>Transaction Status</strong> = 200</p><textarea rows="50" cols="79"><?xml version="1.0" encoding="UTF-8"?>

<Patient xmlns="http://hl7.org/fhir">
  <id value="jan-2019-fhir-experience-example-1005"/>
  <meta>
    <versionId value="10"/>
    <lastUpdated value="2019-01-07T19:54:48.029-05:00"/>
    <profile value="http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"/>
  </meta>
  <text>
    <status value="generated"/>
    <div xmlns="http://www.w3.org/1999/xhtml">
      <p>
        <b>Generated Narrative with Details</b>
      </p>
      <p>
        <b>identifier</b>: 1258622      
      </p>
      <p>
        <b>active</b>: true      
      </p>
      <p>
        <b>name</b>: An P Nguyen       
      </p>
      <p>
        <b>gender</b>: MALE      
      </p>
      <p>
        <b>birthDate</b>: Oct 15, 1990      
      </p>
      <h3>Communications</h3>
      <table class="grid">
        <tr>
          <td>-</td>
          <td>
            <b>Language</b>
          </td>
        </tr>
        <tr>
          <td>*</td>
          <td>English             
            <span style="background: LightGoldenRodYellow">(Details : {urn:ietf:bcp:47 code &#39;en&#39; = &#39;en&#39;, given as &#39;English&#39;})</span>
          </td>
        </tr>
      </table>
    </div>
  </text>
  <extension url="http://hl7.org/fhir/us/core/StructureDefinition/us-core-birthsex">
    <valueCode value="M"/>
  </extension>
  <extension url="http://hl7.org/fhir/us/core/StructureDefinition/us-core-race">
    <extension url="text">
      <valueString value="Asian"/>
    </extension>
    <extension url="ombCategory">
      <valueCoding>
        <system value="urn:oid:2.16.840.1.113883.6.238"/>
        <code value="2028-9"/>
        <display value="Asian"/>
      </valueCoding>
    </extension>
  </extension>
  <extension url="http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity">
    <extension url="text">
      <valueString value="Asian"/>
    </extension>
    <extension url="ombCategory">
      <valueCoding>
        <system value="urn:oid:2.16.840.1.113883.6.238"/>
        <code value="2186-5"/>
        <display value="Non Hispanic or Latino"/>
      </valueCoding>
    </extension>
  </extension>
  <identifier>
    <system value="http://example.org/pids"/>
    <value value="1258622"/>
  </identifier>
  <active value="true"/>
  <name>
    <family value="Nguyen"/>
    <given value="An"/>
    <given value="P"/>
  </name>
  <gender value="male"/>
  <birthDate value="1990-10-15"/>
  <communication>
    <language>
      <coding>
        <system value="urn:ietf:bcp:47"/>
        <code value="en"/>
        <display value="English"/>
      </coding>
      <text value="English"/>
    </language>
  </communication>
</Patient></textarea>

### Extending this example

There are several ways to improve the above code:

For example

- Checking for missing data
- adding capability for repeating elements
- refactoring code to make more compact and reusable